<a href="https://colab.research.google.com/github/onlyabhilash/Spark-NLP_NER_BERT/blob/main/Spark_nlp_NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.7.5
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

spark = sparknlp.start(spark23=True)

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
     |████████████████████████████████| 139 kB 8.6 MB/s 


In [2]:
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.7.5
Apache Spark version:  2.3.0


In [3]:
spark = sparknlp.start(gpu=True)

In [4]:
from urllib.request import urlretrieve

urlretrieve('https://github.com/JohnSnowLabs/spark-nlp/raw/master/src/test/resources/conll2003/eng.train',
           'eng.train')

urlretrieve('https://github.com/JohnSnowLabs/spark-nlp/raw/master/src/test/resources/conll2003/eng.testa',
           'eng.testa')

('eng.testa', <http.client.HTTPMessage at 0x7f83554f2ed0>)

In [7]:
with open('eng.testa') as f:
  c = f.read()

print(c[:1000])

-DOCSTART- -X- -X- O

CRICKET NNP B-NP O
- : O O
LEICESTERSHIRE NNP B-NP B-ORG
TAKE NNP I-NP O
OVER IN B-PP O
AT NNP B-NP O
TOP NNP I-NP O
AFTER NNP I-NP O
INNINGS NNP I-NP O
VICTORY NN I-NP O
. . O O

LONDON NNP B-NP B-LOC
1996-08-30 CD I-NP O

West NNP B-NP B-MISC
Indian NNP I-NP I-MISC
all-rounder NN I-NP O
Phil NNP I-NP B-PER
Simmons NNP I-NP I-PER
took VBD B-VP O
four CD B-NP O
for IN B-PP O
38 CD B-NP O
on IN B-PP O
Friday NNP B-NP O
as IN B-PP O
Leicestershire NNP B-NP B-ORG
beat VBD B-VP O
Somerset NNP B-NP B-ORG
by IN B-PP O
an DT B-NP O
innings NN I-NP O
and CC O O
39 CD B-NP O
runs NNS I-NP O
in IN B-PP O
two CD B-NP O
days NNS I-NP O
to TO B-VP O
take VB I-VP O
over IN B-PP O
at IN B-PP O
the DT B-NP O
head NN I-NP O
of IN B-PP O
the DT B-NP O
county NN I-NP O
championship NN I-NP O
. . O O

Their PRP$ B-NP O
stay NN I-NP O
on IN B-PP O
top NN B-NP O
, , O O
though RB B-ADVP O
, , O O
may MD B-VP O
be VB I-VP O
short-lived JJ B-ADJP O
as IN B-PP O
title NN B-NP O
rivals NNS

In [ ]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng.train')
training_data.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[[document, 0, 47...|[[document, 0, 47...|[[token, 0, 1, EU...|[[pos, 0, 1, NNP,...|[[named_entity, 0...|
|     Peter Blackburn|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 4, Pe...|[[pos, 0, 4, NNP,...|[[named_entity, 0...|
| BRUSSELS 1996-08-22|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, BR...|[[pos, 0, 7, NNP,...|[[named_entity, 0...|
|The European Comm...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ...|[[named_entity, 0...|
|Germany 's repres...|[[document, 0, 21...|[[document, 0, 21...|[[token, 0, 6, Ge...|[[pos, 0, 6, NNP,..

In [ ]:
training_data.count()

14041

In [ ]:
# we use BERT Tiny
#from flair.embeddings import BertEmbeddings

bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [ ]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = bert_annotator.transform(test_data)

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|CRICKET - LEICEST...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 6, CR...|[[pos, 0, 6, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|   LONDON 1996-08-30|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 5, LO...|[[pos, 0, 5, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|West Indian all-r...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 3, We...|[[pos, 0, 3, NNP,...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
# let's transform and save our test dataset for evaluation
test_data.write.parquet("test_withEmbeds.parquet")

In [ ]:
test_data.select("bert.result","bert.embeddings",'label.result').show()

+--------------------+--------------------+--------------------+
|              result|          embeddings|              result|
+--------------------+--------------------+--------------------+
|[cricket, -, leic...|[[-1.6099558, 0.5...|[O, O, B-ORG, O, ...|
|[london, 1996-08-30]|[[-0.6607425, 0.7...|          [B-LOC, O]|
|[west, indian, al...|[[-1.2108904, 0.9...|[B-MISC, I-MISC, ...|
|[their, stay, on,...|[[-0.93976367, 0....|[O, O, O, O, O, O...|
|[after, bowling, ...|[[-1.1267804, 1.1...|[O, O, B-ORG, O, ...|
|[trailing, by, 21...|[[-1.8359257, 0.4...|[O, O, O, O, B-OR...|
|[essex, ,, howeve...|[[-1.2150193, 0.2...|[B-ORG, O, O, O, ...|
|[hussain, ,, cons...|[[-1.6078956, 0.5...|[B-PER, O, O, O, ...|
|[by, the, close, ...|[[-1.8683752, 1.1...|[O, O, O, B-ORG, ...|
|[at, the, oval, ,...|[[-1.8740958, 0.6...|[O, O, B-LOC, O, ...|
|[he, was, well, b...|[[-1.6607139, 1.2...|[O, O, O, O, O, B...|
|[derbyshire, kept...|[[-1.1823792, 0.2...|[B-ORG, O, O, O, ...|
|[australian, tom,...|[[-

In [ ]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(5)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setTestDataset("test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

##### You can also set learning rate ( setLr ), learning rate decay coefficient ( setPo ), setBatchSize and setDropout rate. Please see the official APIs for the entire list.

In [ ]:
%%time

ner_model = pipeline.fit(training_data)

CPU times: user 6.61 s, sys: 680 ms, total: 7.29 s
Wall time: 19min 45s


In [ ]:
%pwd

'/content'

In [ ]:
!ls -l /root/annotator_logs/

total 8
-rw-r--r-- 1 root root 4210 Mar 10 12:35 NerDLApproach_44638d31d89ca7f28f8a.log


In [ ]:
!cd - && cd annotator_logs && ls -l

/
/bin/bash: line 0: cd: annotator_logs: No such file or directory


In [ ]:
%cd

/root


In [ ]:
!cat /root/annotator_logs/NerDLApproach_16a0e7b3577f.log

cat: /root/annotator_logs/NerDLApproach_16a0e7b3577f.log: No such file or directory


In [ ]:
# let's save our trained NER model on disk
# so we can load it in a new session or move it to another location
# since we fit NerDL model inside the pipeline, we can access it via stages
ner_model.stages[1].write().overwrite().save('./NER_bert_20200219')

In [ ]:
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|CRICKET - LEICEST...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 6, CR...|[[pos, 0, 6, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|   LONDON 1996-08-30|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 5, LO...|[[pos, 0, 5, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|West Indian all-r...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 3, We...|[[pos, 0, 3, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|Their stay on top...|[[document, 0, 20...|[[document, 0, 20...|[[token, 0, 4, Th...|[[pos, 0, 4, PRP$...|

In [ ]:
# let's only feed sentence and token from our test dataset
predictions = ner_model.transform(test_data.select("sentence", "token", "label"))
predictions.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            sentence|               token|               label|                bert|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[[document, 0, 64...|[[token, 0, 6, CR...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 16...|[[token, 0, 5, LO...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 18...|[[token, 0, 3, We...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
predictions.select('token.result','label.result','ner.result').show(truncate = 40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                  result|                                  result|                                  result|
+----------------------------------------+----------------------------------------+----------------------------------------+
|[CRICKET, -, LEICESTERSHIRE, TAKE, OV...|   [O, O, B-ORG, O, O, O, O, O, O, O, O]|   [O, O, B-ORG, O, O, O, O, O, O, O, O]|
|                    [LONDON, 1996-08-30]|                              [B-LOC, O]|                              [B-LOC, O]|
|[West, Indian, all-rounder, Phil, Sim...|[B-MISC, I-MISC, O, B-PER, I-PER, O, ...|[B-MISC, I-MISC, O, B-PER, I-PER, O, ...|
|[Their, stay, on, top, ,, though, ,, ...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|
|[After, bowling, Somerset, out, for, ...|[O, O, B-ORG, O, O, O, O, O, O, O, O,...|[O, B-ORG, I-ORG, O, O, O, O, O, O, O...|


In [ ]:
predictions.printSchema()

root
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (n

In [ ]:
import pyspark.sql.functions as F

#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
  #.select(F.expr("cols['0']").alias("token"),
   #     F.expr("cols['1']").alias("ground_truth"),
    #    F.expr("cols['2']").alias("prediction")).show(truncate=False)

### Convert to Pandas

In [ ]:
import pandas as pd

df = predictions.select('token.result','label.result','ner.result').toPandas()

df

,result,result,result
0,"[CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, T...","[O, O, B-ORG, O, O, O, O, O, O, O, O]","[O, O, B-ORG, O, O, O, O, O, O, O, O]"
1,"[LONDON, 1996-08-30]","[B-LOC, O]","[B-LOC, O]"
2,"[West, Indian, all-rounder, Phil, Simmons, too...","[B-MISC, I-MISC, O, B-PER, I-PER, O, O, O, O, ...","[B-MISC, I-MISC, O, B-PER, I-PER, O, O, O, O, ..."
3,"[Their, stay, on, top, ,, though, ,, may, be, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,..."
4,"[After, bowling, Somerset, out, for, 83, on, t...","[O, O, B-ORG, O, O, O, O, O, O, O, O, B-LOC, I...","[O, B-ORG, I-ORG, O, O, O, O, O, O, O, O, B-LO..."
...,...,...,...
3245,"[But, the, prices, may, move, in, a, close, ra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3246,"[Brokers, said, blue, chips, like, IDLC, ,, Ba...","[O, O, O, O, O, B-ORG, O, B-ORG, I-ORG, O, B-O...","[O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O,..."
3247,"[They, said, there, was, still, demand, for, b...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3248,"[The, DSE, all, share, price, index, closed, 2...","[O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O,...","[O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O,..."


### Prediction Pipeline

In [ ]:
from pyspark.ml import Pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(True)

loaded_ner_model = NerDLModel.load("NER_bert_20200219")\
 .setInputCols(["sentence", "token", "bert"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter])

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [ ]:
empty_data = spark.createDataFrame([['']]).toDF('text')
empty_data.show()

+----+
|text|
+----+
|    |
+----+



In [ ]:
prediction_model = ner_prediction_pipeline.fit(empty_data)

In [ ]:
text = "Peter Parker is a nice guy and lives in New York."
sample_data = spark.createDataFrame([[text]]).toDF("text")
sample_data.show()

+--------------------+
|                text|
+--------------------+
|Peter Parker is a...|
+--------------------+



In [ ]:
preds = prediction_model.transform(sample_data)

preds.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|                text|            document|            sentence|               token|                bert|                 ner|ner_span|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|Peter Parker is a...|[[document, 0, 48...|[[document, 0, 48...|[[token, 0, 4, Pe...|[[word_embeddings...|[[named_entity, 0...|      []|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+



In [ ]:
preds.select('ner_span.result').take(1)

[Row(result=[])]

In [ ]:
#preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
#.select(F.expr("entities['0']").alias("chunk"),
 #       F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

### Pretrained Pipelines

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

pretrained_pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

#onto_recognize_entities_sm
#explain_document_dl

recognize_entities_dl download started this may take some time.
Approx size to download 159 MB
[OK!]


In [ ]:
text = "The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."

result = pretrained_pipeline.annotate(text)

list(zip(result['token'], result['ner']))


[('The', 'O'),
 ('Mona', 'B-PER'),
 ('Lisa', 'I-PER'),
 ('is', 'O'),
 ('a', 'O'),
 ('16th', 'O'),
 ('century', 'O'),
 ('oil', 'O'),
 ('painting', 'O'),
 ('created', 'O'),
 ('by', 'O'),
 ('Leonardo', 'B-PER'),
 ('.', 'O'),
 ("It's", 'O'),
 ('held', 'O'),
 ('at', 'O'),
 ('the', 'O'),
 ('Louvre', 'B-LOC'),
 ('in', 'O'),
 ('Paris', 'B-LOC'),
 ('.', 'O')]

In [ ]:
pretrained_pipeline2 = PretrainedPipeline('explain_document_dl', lang='en')

explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[OK!]


In [ ]:
text = "The Mona Lisa is a 16th centry oil painting created by Leonrdo. It's held at the Louvre in Paris."

result2 = pretrained_pipeline2.annotate(text)

result2
list(zip(result2['token'],  result2['checked'], result2['pos'], result2['ner'],  result2['lemma'],  result2['stem']))

[('The', 'The', 'DT', 'O', 'The', 'the'),
 ('Mona', 'Mona', 'NNP', 'B-PER', 'Mona', 'mona'),
 ('Lisa', 'Lisa', 'NNP', 'I-PER', 'Lisa', 'lisa'),
 ('is', 'is', 'VBZ', 'O', 'be', 'i'),
 ('a', 'a', 'DT', 'O', 'a', 'a'),
 ('16th', '6th', 'CD', 'O', '6th', '6th'),
 ('centry', 'centry', 'NN', 'O', 'centry', 'centri'),
 ('oil', 'oil', 'NN', 'O', 'oil', 'oil'),
 ('painting', 'painting', 'NN', 'O', 'painting', 'paint'),
 ('created', 'created', 'VBN', 'O', 'create', 'creat'),
 ('by', 'by', 'IN', 'O', 'by', 'by'),
 ('Leonrdo', 'Leonardo', 'NNP', 'B-ORG', 'Leonardo', 'leonardo'),
 ('.', '.', '.', 'O', '.', '.'),
 ("It's", "it's", 'JJ', 'O', "it's", "it'"),
 ('held', 'held', 'NN', 'O', 'hold', 'held'),
 ('at', 'at', 'IN', 'O', 'at', 'at'),
 ('the', 'the', 'DT', 'O', 'the', 'the'),
 ('Louvre', 'Louvre', 'NNP', 'B-LOC', 'Louvre', 'louvr'),
 ('in', 'in', 'IN', 'O', 'in', 'in'),
 ('Paris', 'Paris', 'NNP', 'B-LOC', 'Paris', 'pari'),
 ('.', '.', '.', 'O', '.', '.')]

In [ ]:
#xx= pretrained_pipeline2.fullAnnotate(text)

#[(n.result, n.metadata['entity']) for n in xx['ner_span']]

### with Glove Embeddings

In [ ]:
glove = WordEmbeddingsModel().pretrained() \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("glove")\
 .setCaseSensitive(False)

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = glove.transform(test_data.limit(1000))

test_data.write.parquet("test_withGloveEmbeds.parquet")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
nerTagger.setInputCols(["sentence", "token", "glove"])
nerTagger.setTestDataset("test_withGloveEmbeds.parquet")

glove_pipeline = Pipeline(
    stages = [
    glove,
    nerTagger
  ])

In [ ]:
%%time

ner_model_v3 = glove_pipeline.fit(training_data.limit(1000))

CPU times: user 475 ms, sys: 63.8 ms, total: 539 ms
Wall time: 1min 22s


In [ ]:
predictions_v3 = ner_model_v3.transform(test_data.limit(10))

# test_data.sample(False,0.1,0)

In [ ]:
#predictions_v3.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
 #       F.expr("cols['1']").alias("ground_truth"),
  #      F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [ ]:
import numpy as np
np.array (predictions.select('token.result').take(1))[0][0]

array(['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP',
       'AFTER', 'INNINGS', 'VICTORY', '.'], dtype='<U14')

In [ ]:
import pandas as pd

tokens = np.array (predictions.select('token.result').take(1))[0][0]
ground = np.array (predictions.select('label.result').take(1))[0][0]
label_bert_0 = np.array (predictions.select('ner.result').take(1))[0][0]
#label_bert_2 = np.array (predictions_v2.select('ner.result').take(1))[0][0]
label_glove = np.array (predictions_v3.select('ner.result').take(1))[0][0]

pd.DataFrame({'token':tokens,
              'ground':ground,
              'label_bert_0':label_bert_0,
              #'label_bert_2':label_bert_2,
              'label_glove':label_glove})

,token,ground,label_bert_0,label_glove
0,CRICKET,O,O,O
1,-,O,O,O
2,LEICESTERSHIRE,B-ORG,O,B-ORG
3,TAKE,O,O,O
4,OVER,O,O,O
5,AT,O,O,O
6,TOP,O,O,O
7,AFTER,O,O,O
8,INNINGS,O,O,O
9,VICTORY,O,O,O


### Using your own custom Word Embedding

In [ ]:
custom_embeddings = WordEmbeddings()\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("glove")\
  .setStoragePath('/Users/vkocaman/cache_pretrained/PubMed-shuffle-win-2.bin', "BINARY")\
.setDimension(200)

In [ ]:
custom_embe = custom_embeddings.fit(training_data.limit(10))

In [ ]:
predictions_v4 = custom_embe.transform(test_data.limit(10))

In [ ]:
#predictions_v4.show()

### creating your own CoNLL dataset

In [ ]:
import json
import os
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

spark = sparknlp.start()

def get_ann_pipeline ():
    
    document_assembler = DocumentAssembler() \
        .setInputCol("text")\
        .setOutputCol('document')

    sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')\
        .setCustomBounds(['\n'])

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    pos = PerceptronModel.pretrained() \
              .setInputCols(["sentence", "token"]) \
              .setOutputCol("pos")
    
    embeddings = WordEmbeddingsModel.pretrained()\
          .setInputCols(["sentence", "token"])\
          .setOutputCol("embeddings")

    ner_model = NerDLModel.pretrained() \
          .setInputCols(["sentence", "token", "embeddings"]) \
          .setOutputCol("ner")

    ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")

    ner_pipeline = Pipeline(
        stages = [
            document_assembler,
            sentence,
            tokenizer,
            pos,
            embeddings,
            ner_model,
            ner_converter
        ]
    )

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    ner_pipelineFit = ner_pipeline.fit(empty_data)

    ner_lp_pipeline = LightPipeline(ner_pipelineFit)

    print ("Spark NLP NER lightpipeline is created")

    return ner_lp_pipeline

In [ ]:
conll_pipeline = get_ann_pipeline ()

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]
Spark NLP NER lightpipeline is created


In [ ]:
parsed = conll_pipeline.annotate ("Peter Parker is a nice guy and lives in New York.")
parsed

{'document': ['Peter Parker is a nice guy and lives in New York.'],
 'embeddings': ['Peter',
  'Parker',
  'is',
  'a',
  'nice',
  'guy',
  'and',
  'lives',
  'in',
  'New',
  'York',
  '.'],
 'ner': ['B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'O'],
 'ner_chunk': ['Peter Parker', 'New York'],
 'pos': ['NNP',
  'NNP',
  'VBZ',
  'DT',
  'JJ',
  'NN',
  'CC',
  'NNS',
  'IN',
  'NNP',
  'NNP',
  '.'],
 'sentence': ['Peter Parker is a nice guy and lives in New York.'],
 'token': ['Peter',
  'Parker',
  'is',
  'a',
  'nice',
  'guy',
  'and',
  'lives',
  'in',
  'New',
  'York',
  '.']}

In [ ]:
conll_lines=''

for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

    conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)


print(conll_lines)

Peter NNP NNP B-PER
Parker NNP NNP I-PER
is VBZ VBZ O
a DT DT O
nice JJ JJ O
guy NN NN O
and CC CC O
lives NNS NNS O
in IN IN O
New NNP NNP B-LOC
York NNP NNP I-LOC
. . . O

